In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# print(len(pd.read_csv('../Data/CleanData/Unified/nacUnified.csv',sep=',',dtype='object').columns))
# print(len(pd.read_csv('../Data/CleanData/Unified/fetalUnified.csv',sep=',',dtype='object').columns))
# print(len(pd.read_csv('../Data/CleanData/Unified/nofetalUnified.csv',sep=',',dtype='object').columns))

In [ ]:
nac = pd.read_csv('../Data/CleanData/Unified/nacUnified.csv',sep=',')
fet = pd.read_csv('../Data/CleanData/Unified/fetalUnified.csv',sep=',')
nofet = pd.read_csv('../Data/CleanData/Unified/nofetalUnified.csv',sep=',') 

## Rename some columns: Change (AREANAC, A_DEFUN,...) -> AREA

Convencion: todas variables se refieren al parto (lugar, mes, etc) a menos que se especifique

In [ ]:
nac.info()

In [ ]:
nac.columns

In [ ]:
nac.columns = ['IGNORE','COD_DPTO', 'COD_MUNIC', 'AREA', 'SIT', 'OTRO_SIT', 'NOM_INST',
     'COD_INST','SEXO', 'PESO', 'TALLA', 'ANO', 'MES', 'PARTO_ATENDIDO_POR', 
     'PARTO_ATENDIDO_POR_OTRO','TIEMPO_GESTACION', 'NUM_CONSUL','TIPO_PARTO', 'TIPO_EMBARAZO', 'APGAR1', 
     'APGAR2', 'IDHEMOCLAS', 'IDFACTORRH', 'IDPUEBLOIN', 'EDAD_MADRE', 
     'ESTADO_CIVIL_M', 'NIV_EDUCATIVO_M','ULT_GRADO_MADRE', 'PAIS_RESIDENCIA', 'DEPTO_RESIDENCIA', 'MUN_RECIDENCIA', 
     'AREA_RESIDENCIA_HAB', 'N_HIJOSV','FECHA_NAC_HIJO_VIVO', 'NUMERO_EMBARAZOS', 'SEG_SOCIAL', 
     'TIPO_ADM_SALUD', 'NOMBRE_ADM_SALUD','EDAD_PADRE','NIV_EDUCATIVO_P', 'ULT_GRADO_PADRE',
     'PROFESION_QUIEN_CERTIFICA', 'FILE_YEAR', 'ORIGINAL_FILE', 'IDPERTET']

fet.columns = ['IGNORE','COD_DPTO', 'COD_MUNIC', 'AREA', 'SIT', 'OTRO_SIT',
        'COD_INST', 'NOM_INST','RESULTADO_EMB', 'ANO', 'MES', 'HORA', 'MINUTOS', 
        'SEXO', 'PAIS_RESIDENCIA', 'DEPTO_RESIDENCIA', 'MUN_RECIDENCIA', 'AREA_RESIDENCIA_HAB', 'SEG_SOCIAL',
        'TIPO_ADM_SALUD', 'NOMBRE_ADM_SALUD', 'MANERA_MUERTE', 'CERT_EXPEDIDO_POR', 'MOMENTO_MUERTE', 
        'TIPO_PARTO', 'TIPO_EMBARAZO', 'TIEMPO_GESTACION', 'PESO', 'EDAD_MADRE', 'N_HIJOSV',
        'N_HIJOSM', 'ESTADO_CIVIL_M','NIV_EDUCATIVO_M', 'ULT_GRADO_MADRE', 'MANERA_MUERTE_VIOLENTA', 'CODOCUR', 
        'CODMUNOC', 'C_MUERTEA', 'C_MUERTEB', 'C_MUERTEC', 'C_MUERTED', 
        'C_MUERTEE', 'ASIS_MED', 'C_DIR1', 'C_DIR12', 'C_ANT1', 'C_ANT12',
        'C_ANT2', 'C_ANT22', 'C_ANT3', 'C_ANT32', 'C_PAT1', 'C_PAT2', 'C_BAS1', 
        'C_MCM1', 'PROFESION_QUIEN_CERTIFICA', 'CAUSA_666', 'CAU_HOMOL', 'FILE_YEAR',
        'ORIGINAL_FILE', 'IDADMISALUD', 'CAUSA_667']

nofet.columns = ['IGNORE','COD_DPTO', 'COD_MUNIC', 'AREA', 'SIT', 'OTRO_SIT',
        'COD_INST', 'NOM_INST','RESULTADO_EMB', 'ANO', 'MES', 'HORA', 'MINUTOS', 
        'SEXO', 'EST_CIVIL', 'GRU_ED1', 'GRU_ED2', 'NIVEL_EDU_FALLECIDO', 'ULTCURFAL', 
        'MUERTEPORO', 'SIMUERTEPO', 'OCUPACION', 'IDPERTET', 'PAIS_RESIDENCIA', 
        'DEPTO_RESIDENCIA', 'MUN_RECIDENCIA', 'AREA_RESIDENCIA_HAB', 'SEG_SOCIAL', 'TIPO_ADM_SALUD', 
        'NOMBRE_ADM_SALUD', 'MANERA_MUERTE', 'CERT_EXPEDIDO_POR', 'MOMENTO_MUERTE', 'TIPO_PARTO', 
        'TIPO_EMBARAZO', 'TIEMPO_GESTACION', 'PESO', 'EDAD_MADRE', 'N_HIJOSV', 'N_HIJOSM',
        'ESTADO_CIVIL_M','NIV_EDUCATIVO_M', 'ULT_GRADO_MADRE', 'EMB_FAL', 'EMB_SEM', 'EMB_MES',
        'MANERA_MUERTE_VIOLENTA', 'CODOCUR', 'CODMUNOC', 'C_MUERTEA', 'C_MUERTEB', 'C_MUERTEC',
        'C_MUERTED', 'C_MUERTEE', 'ASIS_MED', 'C_DIR1', 'C_DIR12', 'C_ANT1', 
        'C_ANT12', 'C_ANT2', 'C_ANT22', 'C_ANT3', 'C_ANT32', 'C_PAT1', 'C_PAT2', 
        'C_BAS1', 'C_MCM1','PROFESION_QUIEN_CERTIFICA', 'CAUSA_666', 'CAU_HOMOL', 'FILE_YEAR',
       'ORIGINAL_FILE', 'IDPUEBLOIN', 'IDADMISALUD', 'CAUSA_667']
                 
nac['RESULTADO_EMB'] = 3
nac["NACIDO_VIVO"] = 1
fet["NACIDO_VIVO"] = 0
nofet["NACIDO_VIVO"] = 0

In [ ]:
nac.shape , fet.shape , nofet.shape

### El df nofet contiene datos de muertes de mayores de edad. A partir de aquí: seleccionar solo las entradas correspondientes a menores de 1 año. (GRU_ED1 < 07 , GRU_ED2 = 01)

In [ ]:
#Asegurarse que variables son consistentes (deberían tener igual cantidad de entradas para este query)
print(nofet[nofet['GRU_ED2'].astype(int) == 1].shape)
print(nofet[nofet['GRU_ED1'].astype(int) < 7].shape)

In [ ]:
#That checked, slice the df
nofet = nofet[nofet['GRU_ED2'].astype(int) == 1]
nofet.head()



## Replacement dicts


In [ ]:
replacements = {
    
    "TIPO_PARTO": {
        1 : "ESPONTANEO",
        2 : "CESAREA",
        3 : "INSTRUMENTADO",
        4 : "IGNORADO",
        9 : "SIN_INFORMACION"
    },
    
    "SEG_SOCIAL": {
        1 : "CONTRIBUTIVO",
        2 : "SUBSIDIADO",
        3 : "EXCEPCION",
        4 : "ESPECIAL",
        5 : "NO_ASEGURADO",
        9 : "SIN_INFORMACION"
    },
    
    "MANERA_MUERTE": {
        0 : "SIN_INFORMACION",
        1 : "NATURAL",
        2 : "VIOLENTA",
        3 : "EN_ESTUDIO"
    },
    
    "SITIO_EVENTO":{
        1 : 'INSTITUCION_SALUD',
        3 : 'CASA',
        4 : 'TRABAJO',
        5 : 'VIA_PUBLICA',
        6 : 'OTRO',
        9 : 'SIN_INFORMACION'
    },    
    
    "CERT_EXPEDIDO_POR":{
        1 : "MEDICO_TRATANTE",
        2 : "MEDICO_NO_TRATANTE",
        3 : "MEDICO_LEGISTA",
        4 : "PERSONAL_AUTORIZADO",
        5 : "FUNCIONARIO_REG_CIVIL",
        6 : "OTRO"
    },
    
    "MOMENTO_MUERTE" : {
        1 : "ANTES",
        2 : "DURANTE",
        3 : "DESPUES",
        4 : "IGNORADO",
        9 : "SIN_INFORMACION"
    },
    
    "TIPO_EMBARAZO" : {
        1 : "SIMPLE",
        2 : "DOBLE",
        3 : "TRIPLE",
        4 : "CUADRUPLE_MAs",
        5 : "IGNORADO"
    },
    
    "TIEMPO_GESTACION" : {
        1 : "0-22",
        2 : "22-27",
        3 : "28-37",
        4 : "38-41",
        5 : "42-MAS",
        6 : "IGNORADO",
        9 : "SIN_INFORMACION"
    },
    
    "MANERA_MUERTE_VIOLENTA":{
        0 : "NO_VIOLENTA",
        1 : "SUICIDIO",
        2 : "HOMICIDIO",
        3 : "ACCIDENTE_TRANSITO",
        4 : "OTRO_ACCIDENTE",
        5 : "EN_ESTUDIO"
    },
    
    "PARTO_ATENDIDO_POR" : {
        1 : "MEDICO",
        2 : "ENFERMERA",
        3 : "AUXILIAR_ENFERMERIA",
        4 : "PROMOTOR_SALUD",
        5 : "PARTERA",
        6 : "OTRO",
        9 : "SIN_INFORMACION"
    },
    "PROFESION_QUIEN_CERTIFICA" : {
        1 : "MEDICO",
        2 : "ENFERMERA",
        3 : "AUXILIAR_ENFERMERIA",
        4 : "PROMOTOR_SALUD",
        5 : "FUNCIONARIO_REG_CIVIL",
        6 : "MEDICO_LEGISTA",
        9 : "SIN_INFORMACION"
    },
    
    "RESULTADO_EMB" : {
        1 : "DEFUNCION_FETAL",
        2 : "DEFUNCION_NO_FETAL",
        3 : "NACIDO_VIVO"
    },
    
    "SEXO": {
        1 : "MASCULINO",
        2 : "FEMENINO",
        3 : "INDETERMINADO",
    },
    
    "NIV_EDUCATIVO_M":{
         1:"PREESCOLAR",
         2:"PRIMARIA",
         3:"SECUNDARIA",
         4:"SECUNDARIA",
         5:"MEDIA_TECNICA",
         6:"NORMALISTA",
         7:"TECNICA_PROFESIONAL",
         8:"TECNOLOGICA",
         9:"PROFESIONAL",
        10:"ESPECIALIZACION",
        11:"MAESTRIA",
        12:"DOCTORADO",
        13:"NINGUNO",
        99:"SIN_INFORMACION"
    },
    "NIV_EDUCATIVO_P":{
         1:"PREESCOLAR",
         2:"PRIMARIA",
         3:"SECUNDARIA",
         4:"SECUNDARIA",
         5:"MEDIA_TECNICA",
         6:"NORMALISTA",
         7:"TECNICA_PROFESIONAL",
         8:"TECNOLOGICA",
         9:"PROFESIONAL",
        10:"ESPECIALIZACION",
        11:"MAESTRIA",
        12:"DOCTORADO",
        13:"NINGUNO",
        99:"SIN_INFORMACION"
    }
}

# Estandarizar sitio de parto (defunción)

### fet y nofet:
Sitio donde ocurrió la defunción
1 Hospital/clínica
2 Centro /puesto de salud
3 Casa /domicilio
4 Lugar de trabajo
5 Vía pública
6 Otro
9 Sin información

### nac:
Sitio de la Parto
1 = Institución de salud
2 = Domicilio
3 = Otro
9 = Sin información

In [ ]:
#Mezcla hospital y centro de salud en fet y nofet
fet['SIT'].replace('2','1',inplace=True)
nofet['SIT'].replace('2','1',inplace=True)

nac['SIT'].replace('3','6',inplace=True) #otro = 6
nac['SIT'].replace('2','3',inplace=True) #domicilio = 3

dfs = [nac,fet,nofet]

#SITIO EVENTO: nueva columna para poner todos los valores (sumariza SIT y OTRO_SIT)
for df_i in dfs:
    df_i['OTRO_SIT'].replace(' ',9)
    df_i['SITIO_EVENTO'] = df_i['SIT'].astype('object')
    #Reemplaza valores en "replacing"
#     df_i['SITIO_EVENTO'].replace(replacements["SITIO_EVENTO"].keys(),replacements["SITIO_EVENTO"].values(),inplace=True)
    #Simple cleaning
#     df_i['OTRO_SIT'].replace(' ','SIN_INFORMACION',inplace=True) 
#     df_i['OTRO_SIT'] = df_i['OTRO_SIT'].str.upper().str.strip()

### Concatenate all 3 dfs. From now on I assume all data is standarized among dfs

In [ ]:
df = (pd.concat([nac,fet,nofet],sort=True)  
      .replace(' ',np.nan)    #replace missing values
      .reset_index(drop=True)) #reset index and drop old indices
df.shape

In [ ]:
df.info()

In [ ]:
df.columns

## Fill N_EMB, N_HIJOSV and N_HIJOSM. Use N_EMB = N_HIJOSV + N_HIJOSM

In [ ]:
df["N_HIJOSM"].replace('Sin información',99,inplace=True)

#Convert these columns to float
df[['NUMERO_EMBARAZOS','N_HIJOSV','N_HIJOSM']] = df[['NUMERO_EMBARAZOS','N_HIJOSV','N_HIJOSM']].astype(float)

#Print number of missing values in these columns
print('Before:\n',df[['NUMERO_EMBARAZOS','N_HIJOSV','N_HIJOSM']].isna().sum())  

df_tmp = df[df['NUMERO_EMBARAZOS'].isna()]  #df with nan values for N_EMB
df.loc[df['NUMERO_EMBARAZOS'].isna(),['NUMERO_EMBARAZOS']] = df_tmp['N_HIJOSM'] + df_tmp['N_HIJOSV']

df_tmp = df[df['N_HIJOSM'].isna()]  #df with nan values for N_HIJOSM
df.loc[df['N_HIJOSM'].isna(),['N_HIJOSM']] = df_tmp['NUMERO_EMBARAZOS'] - df_tmp['N_HIJOSV']

df_tmp = df[df['N_HIJOSV'].isna()]  #same for N_HIJOSV
df.loc[df['N_HIJOSV'].isna(),['N_HIJOSV']] = df_tmp['NUMERO_EMBARAZOS'] - df_tmp['N_HIJOSM']

df.replace('NO_INFONO_INFO','NO_INFO',inplace=True)
print('\nAfter:\n',df[['NUMERO_EMBARAZOS','N_HIJOSV','N_HIJOSM']].isna().sum()) 

## Create unique code for munic.

### dpto: 05 + munic: 001 => 05001 new code

In [ ]:
#Create unique munic and dpto codes (nacim, resid, death)
def id_municip(df,coldpto,colmunic,nameNew):
    #Convert dpt. and munic. codes to strings and fill with zeros   
    df[coldpto] = df[coldpto].apply(lambda x : str(x).zfill(2))
    df[colmunic] = df[colmunic].apply(lambda x : str(x).zfill(3))
    #Create a single ID for each munic.
    df[nameNew] = df[coldpto] + df[colmunic]
    return df.drop(colmunic,axis=1)  #Drop munic. code, the new code is enough

df = id_municip(df,'COD_DPTO','COD_MUNIC','ID_BIRTH')
df = id_municip(df,'DEPTO_RESIDENCIA', 'MUN_RECIDENCIA','ID_RESID')
df = id_municip(df,'CODOCUR','CODMUNOC','ID_VIOL').replace('nannan',np.nan)

df.shape

In [ ]:
cols = [c for c in df.columns if c.startswith("C_M")]
df[(~df[cols].isnull()).any(axis=1)][cols].head()

In [ ]:
#Unir C_MUERTE(A,B..,E) en una sola columna. Esta variable es como se determinó la causa de muerte
#Tal vez esto no es tan importante, así que las voy a unir y tirar las otras
cols = [f'C_MUERTE{i}' for i in ['A','B','C','D','E']]
df['C_MUERTE'] = df[cols].fillna('_').astype(str).sum(axis=1)
df.drop(cols,axis=1,inplace=True)
df.shape

In [ ]:
df["C_MUERTE"].value_counts()

In [ ]:
#Crear una sola columna para DATE
print(df[['ANO', 'MES', 'HORA', 'MINUTOS']].isna().any())  #Make sure only HORA, MINUTOS are missing

#Convertir data a strings
tmp = df[['ANO', 'MES', 'HORA', 'MINUTOS']].fillna(0).astype(float).astype(int).astype(str)
tmp["HORA"] = tmp["HORA"].replace("98","0").replace("99","0")
tmp["MINUTOS"] = tmp["MINUTOS"].replace("98","0").replace("99","0")
#Sumar con formato Y/M/D H:Min
date = tmp['ANO'] + '/' + tmp['MES'] + ' ' + tmp['HORA'] + ':' + tmp['MINUTOS']   

df['DATE'] = pd.to_datetime(date) #Convert to datetime
df.drop(['ANO', 'MES', 'HORA', 'MINUTOS'],axis=1,inplace=True)  #Drop old columns
df["DATE"].head()

In [ ]:
## For now, fill missing values

#for this vars: no data == 9
vars9 = ['PESO','TALLA','PARTO_ATENDIDO_POR','TIEMPO_GESTACION','TIPO_PARTO','TIPO_EMBARAZO','IDHEMOCLAS', 
         'IDFACTORRH','IDPERTET','ESTADO_CIVIL_M','AREA_RESIDENCIA_HAB','TIPO_ADM_SALUD','ULT_GRADO_PADRE',
         'CERT_EXPEDIDO_POR','MOMENTO_MUERTE','MANERA_MUERTE_VIOLENTA','ASIS_MED','EST_CIVIL',
         'PROFESION_QUIEN_CERTIFICA','IDPUEBLOIN']
for var in vars9:
    df[var].fillna(9,inplace=True)
    
#for this vars: no data == 99
vars99 = ['NUM_CONSUL','APGAR1', 'APGAR2','EDAD_MADRE','NIV_EDUCATIVO_M','N_HIJOSV',
          'NIV_EDUCATIVO_P','N_HIJOSM','NUMERO_EMBARAZOS','PAIS_RESIDENCIA']
for var in vars99:
    df[var].fillna(99,inplace=True)
    
df['ID_VIOL'].fillna('01999',inplace=True)
df['EDAD_PADRE'].fillna(999,inplace=True)
df['MANERA_MUERTE'].fillna(0,inplace=True)

df.loc[:,df.isna().any()].columns

### solo tienen sentido para nofet: GRU_ED1 y 2, NIVEL_EDU_FALLECIDO, ULTCURFAL, MUERTEPORO, SIMUERTEPO, OCUPACION, EMB_FAL, EMB_SEM, EMB_MES

## Let's clean the SIT and OTRO_SIT variables. These indicate where the event (birth, death) happened, and can be joined into a single column. Cleaning of this var needs to be done as there are some spelling mistakes or different expressions for the same place.

In [ ]:
#Let's do some real cleaning

#Si alguna de las palabras en uniq_vals está contenida en "WORD", reemplaza WORD por dicha palabra
uniq_vals = ['AMBUL','CARRO','LANCHA','CARRETERA','VEREDA','TAXI','BUS','PARTERA',
             'TRABAJO','CASA','VIA','LANCHA','CARCEL','BONGO','HOTEL',
             'HOSPITAL','IGLESIA','VEHICULO','DE SALUD','INDIGENA','RESTAURANTE',
             'CLINICA','CALLE','HUERTA','BARRIO','CLINISANITAS',
             'PATRULLA','BANO','RESGUARDO','BOTE','CABILDO','DOMICILIO','V/',
             'ALMACEN','VERDA','FINCA','CANOA','PLAYA','VIVIENDA','PARQUEADERO','RURAL','RIO',
             'INTERPERIE','ALVERGUE','PUENTE','RIO','PUESTODE SALUD','CENTRODE SALUD','IPS','AUTOMOVIL',
             'PANGA','PORTE PUBLICO','SININFORMACION','POLICIA','SERVICIO PUBLICO','FLOTA','MUELLE',
             'CAMIONETA','CAMPO','COMUNIDAD','SIN INFORMACION','GERIATRICO','ANCIAN','GERONTO','HOGAR',
             'ONCOLOGICA','RESIDENCIA','FUNDACION','CAMINO','ASILO','OFICINA','AULA','BOSQUE','PARROQUIA',
             'OTRO','COMERCI','PUBLICO','BAR','LICOR','CULTIVO','DEPORTE','ANSIANATO','TABERNA','CANTINA',
             'FABRICA']

#Reemplaza explicitamente otras ocurrencias
to_replace = {'SIN ESTABLECER':'SIN_INFORMACION','VIA':'CALLE','AVENIDA':'CALLE',
              'AUTOPISTA':'CALLE','V/':'VEREDA','VERDA':'VEREDA','AUTOMOVIL':'VEHICULO',
              'MEDIO DE TRASPORTE ACUATICO (C':'BOTE','BARCO':'BOTE','UTOMOVIL':'VEHICULO',
              'CARRO':'VEHICULO','CANOA':'BOTE','CARETERA':'CALLE','VIVIENDA':'CASA',
              'SIN INFROMACION':'SIN_INFORMACION','AMBUL':'AMBULANCIA','SE DESCONOCE':'SIN_INFORMACION',
              'PANGA':'BOTE',
              'CENTRO DE SLUD':'INSTITUCION DE SALUD','EMBARCACION':'BOTE','CENTRO PENITENCIARIO':'CARCEL',
              'TRANSMILENIO':'TRASPORTE PUBLICO','ALVERGUE':'ALBERGUE','CLINISANITAS':'INSTITUCION DE SALUD',
              'BUS':'TRANSPORTE PUBLICO','CHALUPA':'BOTE','SININFORMACION':'SIN_INFORMACION',
              'SERVICIO PUBLICO':'TRANSPORTE PUBLICO','FLOTA':'TRANSPORTE PUBLICO','CARRETERA':'CALLE',
              'CAMIONETA':'VEHICULO','PORTE PUBLICO':'TRANSPORTE PUBLICO','DESCONOCIDO':'SIN_INFORMACION',
              'DE SALUD':'INSTITUCION DE SALUD','LANCHA':'BOTE','INDIGENA':'COMUNIDAD INDIGENA',
              'DOMICILIO':'CASA','BONGO':'BOTE','RESIDENCIA':'CASA','HOSPITAL':'INSTITUCION DE SALUD',
              'SIN INFORMACION':'SIN_INFORMACION','GERIATRICO':'HOGAR GERIATRICO','ANCIAN':'HOGAR GERIATRICO',
              'GERONTO':'HOGAR GERIATRICO','ONCOLOGICA':'INSTITUCION DE SALUD','PARROQUIA':'IGLESIA',
              'FINCA':'RURAL','VEREDA':'RURAL','COMERCI':'COMERCIO','PUBLICO':'OTRO LUGAR PUBLICO',
              'CULTIVO':'AREA DE CULTIVO','LICOR':'BAR','DEPORTE':'AREA DEPORTE',
              'ANSIANATO':'HOGAR GERIATRICO','TABERNA':'BAR','CANTINA':'BAR'}

def clean_otrosit(df,uniq_vals,to_replace):
    import unicodedata
    def replace_isin(x):
        #Elimina caracteres especiales
        s = ''.join((c for c in unicodedata.normalize('NFD',x) if unicodedata.category(c) != 'Mn'))
        #Convierte a word de uniq_vals
        for word in uniq_vals:
            if word in s:
                return word
        return s

    #Toma solo valores para los que 'SITIO_EVENTO' indique OTRO
    a = df[df['SITIO_EVENTO']=='OTRO']['OTRO_SIT'].copy()
    a = a.apply(lambda x: replace_isin(x))
    a.replace(to_replace.keys(),to_replace.values(),inplace=True) #Reemplaza explicitamente de dict

    init_len = df['OTRO_SIT'].unique().shape[0] #Amount of unique values before replacements
    
    #Set new values
    cp_df = df.copy()
    cp_df.loc[a.index,'SITIO_EVENTO'] = a.values

    final_len = cp_df['SITIO_EVENTO'].unique().shape[0]  #unique values after
    
    #Print summary of cleaning
    print(f"Initial unique values: {init_len}\nFinal unique values: {final_len}")
    
    #Test cleaning. Check everything looks ok
    print(cp_df[cp_df['SITIO_EVENTO']!='INSTITUCION DE SALUD'][['SITIO_EVENTO','OTRO_SIT','SIT']].head(10))

    #Now, we can drop OTRO_SIT AND SIT
    cp_df.drop(columns=['OTRO_SIT','SIT'],inplace=True)
    return cp_df

df = clean_otrosit(df,uniq_vals,to_replace)

## It may be possible to clean some columns that contain no information after the slicing of the nofet df. (ocupacion, nivel edu fallecido, etc)

In [ ]:
#First drop columns that only contain NaNs. I expect this from variables related to pregnancy when dead
# print("Empty columns:")
# print(df.loc[:,df.isna().all()].columns.to_list())

# df = df.loc[:,~df.isna().all()]
print()
### Check other columns that are suspicios of not containing any relevant information.
#GRU_ED2 because it should always be == 1 or NaN
for col in ['EST_CIVIL','GRU_ED2','NIVEL_EDU_FALLECIDO','ULTCURFAL','MANERA_MUERTE_VIOLENTA']:
    print(col ,df[col].unique())

### Can only drop EST_CIVIL, GRU_ED2. Interestingly enough, there is someone in 6th grade (probably a spelling mistake). I'll drop these 2 columns as well

In [ ]:
# df = df.drop(columns=[])

In [ ]:
df.info()

In [ ]:
def replaceValues(df,toNumbers = True):
    dfReplaced = df.copy()
    for field in replacements:
        print(field)
        if(toNumbers):
            dfReplaced[field] = dfReplaced[field].replace(replacements[field].keys(),
                                                          replacements[field].values())            
        else:
            dfReplaced[field] = (dfReplaced[field].astype(float).astype(int)
                                 .replace(replacements[field].keys(),replacements[field].values())) 
    return dfReplaced

df = replaceValues(df,toNumbers=False)

In [ ]:
df["PROFESION_QUIEN_CERTIFICA"].unique()

In [ ]:
df[list(replacements.keys())]

### Let's look at rough percentages of non-informative values

In [ ]:
#Look at percentage of missing values

dic = {}
for i,col in enumerate(df.columns):
    if 'DATE' not in col:
        num_miss = df[df[col].isin([9,99,999,'9','99','999',np.nan,'SIN_INFORMACION'])].shape[0]
        num_tot = df.shape[0]
        dic[col]=[i,num_miss*100/num_tot]

((pd.DataFrame(dic,index=['id','Porcentaje faltantes']).T)
 .sort_values('Porcentaje faltantes',ascending=False).head(60).drop(columns='id'))

In [ ]:
trash = ['EST_CIVIL','GRU_ED2','NIVEL_EDU_FALLECIDO','ULTCURFAL','MUERTEPORO','ULTCURFAL'
         ,'SIMUERTEPO','OCUPACION', 'EMB_FAL', 'EMB_SEM', 'EMB_MES',
         'C_MUERTE','C_ANT1','C_ANT12','C_ANT2','C_ANT22','C_ANT3','MANERA_MUERTE_VIOLENTA',
         'C_BAS1','C_DIR1','C_ANT32','C_DIR12','C_MCM1','C_PAT1','C_PAT2','IGNORE',
        'PARTO_ATENDIDO_POR_OTRO','CAUSA_667','CAUSA_666','CAU_HOMOL','IDADMISALUD']

df.drop(columns=trash,inplace=True)

In [ ]:
df.info()

In [ ]:
colsToConvert = df.dtypes[df.dtypes == np.float64].index
df[colsToConvert]
# df[colsToConvert].astype(int)
# df[colsToConvert] = df[colsToConvert].astype(int)

In [ ]:
for col in colsToConvert:
    print(col,df[col].unique())

In [ ]:
# Falta revisar outliers en 

# NUMERO_EMBARAZOS 
# NUM_CONSUL
# N_HIJOSM 
# N_HIJOSV 

In [ ]:
print(df.columns)
print(df.shape)


### Save in order to reuse this data cleaning later on other notebooks.

In [ ]:
df.to_csv("../Data/CleanData/CleanDB_FULL.csv")

In [ ]:
# df
